### Imports

In [65]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, circuit_drawer
import numpy as np
import matplotlib.pyplot as plt
import heapq
from IPython.display import clear_output
import time

## Define The Maze

#### Maze

In [ ]:
maze = [
    ['S', 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0],
    [1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1],
    [1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0],
    [0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0],
    [1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0],
    [0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0],
    [1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
    [0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0],
    [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0],
    [0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0],
    [1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 'G']
]


#### Define the start and goal positions


In [ ]:
start = (0, 0)  # Top-left corner
goal = (14, 14)   # Bottom-right corner


# Classical Search (A*)

#### Define the Heuristic

In [ ]:
def heuristic(a, b):
    """Heuristic function for A* (Manhattan distance)."""
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

#### Get neighborhood

In [ ]:
def get_neighbors(maze, position):
    """Get valid neighbors for the current position."""
    rows, cols = len(maze), len(maze[0])
    x, y = position
    neighbors = []

    for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:  # Up, Down, Left, Right
        nx, ny = x + dx, y + dy
        if 0 <= nx < rows and 0 <= ny < cols and maze[nx][ny] != 1:  # Check bounds and barriers
            neighbors.append((nx, ny))

    return neighbors


#### Path Construction

In [ ]:
def reconstruct_path(came_from, current):
    """Reconstruct the path from start to goal."""
    path = []
    while current in came_from:
        path.append(current)
        current = came_from[current]
    path.reverse()
    return path


#### A* Algorithm function

In [ ]:
def astar(maze, start, goal):
    """A* algorithm with debug information."""
    open_set = []
    heapq.heappush(open_set, (0, start))
    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic(start, goal)}

    while open_set:
        _, current = heapq.heappop(open_set)

        # Debugging: print current exploration status
        print(f"Exploring: {current}, f_score: {f_score[current]}")

        # Check if the goal is reached
        if current == goal:
            return reconstruct_path(came_from, current)

        for neighbor in get_neighbors(maze, current):
            tentative_g_score = g_score[current] + 1

            if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = g_score[neighbor] + heuristic(neighbor, goal)
                heapq.heappush(open_set, (f_score[neighbor], neighbor))

        # Debugging: print open set status after exploring neighbors
        print(f"Open set: {open_set}")

    return None  # No path found


#### Helper functions

In [ ]:
def show_path(maze, path):
    """
    Visualize the maze with the path.
    - Path cells are marked with '*'.
    - Start is marked with 'S'.
    - Goal is marked with 'G'.
    """
    maze_copy = [row[:] for row in maze]  # Create a copy of the maze to modify

    for x, y in path:
        if maze_copy[x][y] != 'G':  # Avoid overwriting the goal marker
            maze_copy[x][y] = '*'

    start_x, start_y = path[0]
    maze_copy[start_x][start_y] = 'S'  # Mark the start position

    for row in maze_copy:
        print(" ".join(str(cell) for cell in row))


In [ ]:
def print_maze(maze):
    """
    Prints the maze in a human-readable format.
    - Open spaces are shown as '0'.
    - Barriers are shown as '1'.
    - Start is shown as 'S'.
    - Goal is shown as 'G'.
    """
    for row in maze:
        print(" ".join(str(cell) for cell in row))
    print("\n")  # Add a newline for better readability


##### Visualization

In [ ]:
def print_exploration(maze, open_set):
    """
    Visualize the maze during A* exploration.
    - Explored cells in the open_set are marked as 'E'.
    """
    maze_copy = [row[:] for row in maze]
    for _, (x, y) in open_set:
        if maze_copy[x][y] not in ('S', 'G'):  # Avoid overwriting start/goal markers
            maze_copy[x][y] = 'E'

    print_maze(maze_copy)
    time.sleep(0.5)  # Pause for visualization


In [ ]:
def clear_terminal():
    """Clear the output in a Jupyter Notebook."""
    clear_output(wait=True)


#### A* With vizualisation

In [ ]:
def astar_with_visualization(maze, start, goal):
    """
    A* algorithm with visualization of exploration in a Jupyter Notebook.
    """
    open_set = []
    heapq.heappush(open_set, (0, start))
    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic(start, goal)}

    while open_set:
        # Clear the output to update the maze state
        clear_terminal()
        print("Exploring the maze...")

        # Visualize the current state
        print_exploration(maze, open_set)

        _, current = heapq.heappop(open_set)

        # Check if the goal is reached
        if current == goal:
            return reconstruct_path(came_from, current)

        for neighbor in get_neighbors(maze, current):
            tentative_g_score = g_score[current] + 1

            if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = g_score[neighbor] + heuristic(neighbor, goal)
                heapq.heappush(open_set, (f_score[neighbor], neighbor))

    return None  # No path found

#### Execute

In [ ]:
# Run the A* algorithm with visualization
path = astar_with_visualization(maze, start, goal)

# Show the final path if found
if path:
    print("Path found:")
    show_path(maze, path)
else:
    print("No path found.")


# Quantum Search

In [ ]:


# Helper function to create the oracle
def create_oracle(goal_index, num_qubits):
    """Create an oracle that marks the goal index."""
    oracle_circuit = QuantumCircuit(num_qubits, name="Oracle")
    binary_goal = format(goal_index, f"0{num_qubits}b")
    for i, bit in enumerate(reversed(binary_goal)):
        if bit == '0':
            oracle_circuit.x(i)
    oracle_circuit.mcx(list(range(num_qubits - 1)), num_qubits - 1)
    for i, bit in enumerate(reversed(binary_goal)):
        if bit == '0':
            oracle_circuit.x(i)
    return oracle_circuit

# Function to run Grover's algorithm
def grover_maze_search(maze):
    num_cells = len(maze)
    num_qubits = int(np.ceil(np.log2(num_cells)))

    # Create a quantum circuit
    qc = QuantumCircuit(num_qubits, num_qubits)
    qc.h(range(num_qubits))  # Apply Hadamard gates to all qubits

    # Create Oracle and Grover Operator
    oracle = create_oracle(goal_index, num_qubits)
    grover_op = QuantumCircuit(num_qubits, name="Grover")
    grover_op.compose(oracle, inplace=True)

    # Add the diffusion operator (reflection about |+> state)
    grover_op.h(range(num_qubits))
    grover_op.x(range(num_qubits))
    grover_op.h(num_qubits - 1)
    grover_op.mcx(list(range(num_qubits - 1)), num_qubits - 1)
    grover_op.h(num_qubits - 1)
    grover_op.x(range(num_qubits))
    grover_op.h(range(num_qubits))

    # Number of iterations needed
    num_iterations = int(np.floor(np.pi / 4 * np.sqrt(num_cells)))

    for _ in range(num_iterations):
        qc.compose(oracle, inplace=True)
        qc.compose(grover_op, inplace=True)

    # Measure the result
    qc.measure(range(num_qubits), range(num_qubits))

    # Draw the circuit
    print("\nGrover's Algorithm Circuit:")
    display(qc.draw())

    # Run the circuit on a simulator
    simulator = AerSimulator()
    job = simulator.run(qc, shots=1024)
    result = job.result()
    counts = result.get_counts()

    return counts

# Run Grover's algorithm on the maze
result = grover_maze_search(maze)

# Visualize the result

# Decode the result
most_likely = max(result, key=result.get)
found_index = int(most_likely, 2)

print(f"Goal cell found at index: {found_index}")


In [ ]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import os
import time
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

def clear_terminal():
    """Clear the terminal output."""
    os.system('cls' if os.name == 'nt' else 'clear')

def visualize_maze(maze, highlight=None, delay=0.5):
    """Visualize the maze in the terminal."""
    clear_terminal()
    grid_size = int(np.sqrt(len(maze)))
    for i, cell in enumerate(maze):
        if i == highlight:
            print("🔍", end=" ")  # Highlight the current cell being processed
        elif cell == 1:  # Wall
            print("#", end=" ")
        elif cell == 0:  # Empty space
            print(".", end=" ")
        elif cell == "G":  # Goal
            print("G", end=" ")
        else:
            print("?", end=" ")
        if (i + 1) % grid_size == 0:
            print()  # New line for the grid
    print("\n")
    print(f"Highlighting cell: {highlight}")
    time.sleep(delay)

def create_oracle(goal_index, num_qubits):
    """Create an oracle that marks the goal index."""
    oracle = QuantumCircuit(num_qubits, name="Oracle")
    binary_goal = format(goal_index, f'0{num_qubits}b')  # Ensure proper binary representation
    for i, bit in enumerate(reversed(binary_goal)):
        if bit == '0':
            oracle.x(i)
    oracle.mcx(list(range(num_qubits - 1)), num_qubits - 1)
    for i, bit in enumerate(reversed(binary_goal)):
        if bit == '0':
            oracle.x(i)
    return oracle


def create_diffusion_operator(num_qubits):
    """Create the Grover diffusion operator."""
    diffusion = QuantumCircuit(num_qubits, name="Diffusion")
    diffusion.h(range(num_qubits))
    diffusion.x(range(num_qubits))
    diffusion.h(num_qubits - 1)
    diffusion.mcx(list(range(num_qubits - 1)), num_qubits - 1)
    diffusion.h(num_qubits - 1)
    diffusion.x(range(num_qubits))
    diffusion.h(range(num_qubits))
    return diffusion

# Grover Search with Improved Visualization
def grover_search_maze(maze, goal_index):
    num_cells = len(maze)
    num_qubits = int(np.ceil(np.log2(num_cells)))
    qc = QuantumCircuit(num_qubits, num_qubits)
    qc.h(range(num_qubits))

    oracle = create_oracle(goal_index, num_qubits)
    diffusion = create_diffusion_operator(num_qubits)
    num_iterations = int(np.floor(np.pi / 4 * np.sqrt(num_cells)))

    for _ in range(num_iterations):
        qc.compose(oracle, inplace=True)
        qc.compose(diffusion, inplace=True)

    qc.measure(range(num_qubits), range(num_qubits))

    simulator = AerSimulator()
    compiled_circuit = transpile(qc, simulator)
    job = simulator.run(compiled_circuit, shots=1024)
    result = job.result()
    counts = result.get_counts()

    # Highlight most probable state
    most_probable_state = max(counts, key=counts.get)
    highlight = int(most_probable_state, 2)
    visualize_maze(maze, highlight=highlight, delay=1)

    return counts


# Define the maze (0 = empty, 1 = wall, "G" = goal)
maze = [0, 0, 0, 1,  # 4x4 grid
        1, 0, 0, 0,
        0, 0, 0, 0,
        1, 1, "G", 1]

goal_index = 14  # Index of the goal cell

# Run Grover's algorithm on the maze
result = grover_search_maze(maze, goal_index)


In [ ]:
try:
    result = grover_search_maze(maze, goal_index)
    plot_histogram(result)

    # Decode the result
    most_likely = max(result, key=result.get)
    found_index = int(most_likely, 2)

    # Visualize the final result
    visualize_maze(maze, delay=0)
    print(f"Goal cell found at index: {found_index} (🔍)")
except ValueError as e:
    print(f"Error: {e}")

In [ ]:
plot_histogram(result)

In [ ]:
# Test Oracle
oracle = create_oracle(goal_index=14, num_qubits=4)
print("Oracle Circuit:")
print(oracle)

# Test Diffusion Operator
diffusion = create_diffusion_operator(num_qubits=4)
print("Diffusion Circuit:")
print(diffusion)

# Test Full Circuit Without Visualization
qc = QuantumCircuit(4, 4)
qc.h(range(4))
qc.compose(oracle, inplace=True)
qc.compose(diffusion, inplace=True)
qc.measure(range(4), range(4))

simulator = AerSimulator()
compiled_circuit = transpile(qc, simulator)
job = simulator.run(compiled_circuit, shots=1024)
result = job.result()
counts = result.get_counts()
print("Final Counts:", counts)
